In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# GCP上的端到端机器学习：MLOps阶段6：使用优化的TensorFlow Enterprise容器与Vertex AI Prediction /文本模型开始

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_optimized_tfe_bert.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_optimized_tfe_bert.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage6/get_started_with_optimized_tfe_bert.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>       
</table>
<br/><br/><br/>

## 概述

本教程演示了如何将 TensorFlow 文本模型上传并部署到带有 TensorFlow 企业（TFE）运行时优化容器的 `Vertex AI Endpoint`。

虽然优化可以应用于任何 TensorFlow 模型，但优化对表格和文本模型类型效果最好。

*注意:* 使用 T4 训练模型大约需要 15 分钟。如果在 CPU 上训练，需要几个小时。

### 目标

在本教程中，您将学习如何使用`TensorFlow Enterprise Optimized`容器部署到`Vertex AI Endpoint`资源的TensorFlow模型。

本教程使用以下Google Cloud ML服务和资源：

- `Vertex AI Prediction`
- `Vertex AI Models`
- `Vertex AI Endpoints`
- `TensorFlow Enterprise Optimized`容器

执行的步骤包括：

- 从TensorFlow Hub下载一个预训练的BERT模型。
- 将BERT模型进行微调（迁移学习）作为二元分类器。
- 将TensorFlow Hub模型上传为`Vertex AI Model`资源，使用标准的TensorFlow serving容器。
- 将TensorFlow Hub模型上传为`Vertex AI Model`资源，使用TensorFlow Enterprise Optimized容器。
- 创建两个`Endpoint`资源。
- 将两个`Model`资源部署到单独的`Endpoint`资源中。
- 对部署的两个`Model`资源进行相同的在线预测请求。
- 比较两个部署的`Model`资源之间的预测准确性。
- 配置容器设置以控制微调优化。
- 创建一个`Private Endpoint`资源。
- 将使用`TensorFlow Enterprise Optimized`的`Model`资源部署到`Private Endpoint`资源中。
- 向`Private Endpoint`资源发出在线预测请求。 

了解更多关于[TensorFlow Enterprise Optimized container](https://cloud.google.com/vertex-ai/docs/predictions/optimized-tensorflow-runtime)

### 数据集

本教程使用了来自TensorFlow Hub的一个预训练的BERT文本模型，然后在包含50,000部电影评论的[大型电影评论数据集](https://ai.stanford.edu/~amaas/data/sentiment/)（来自[互联网电影数据库](https://www.imdb.com/)）上进行微调（迁移学习）。训练好的模型可以预测一部电影评论是积极的还是消极的。

了解更多关于[BERT预训练编码器模型](https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3)。

成本

本教程使用Google Cloud的计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI
价格](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage
价格](https://cloud.google.com/storage/pricing)，并使用[Pricing
Calculator](https://cloud.google.com/products/calculator/)
根据您预期的使用情况生成成本估算。

## 安装

安装以下包以运行此笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG -q
! pip3 install --upgrade google-cloud-pipeline-components $USER_FLAG -q
! pip3 install tensorflow tensorflow-text $USER_FLAG -q
! pip3 install tensorflow-hub $USER_FLAG -q
! pip3 install tf-models-official $USER_FLAG -q

### 重新启动内核

安装额外的包后，您需要重新启动笔记本内核，以便它可以找到这些包。

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

##在开始之前

### GPU运行时

*如果您有这个选项，请确保您在GPU运行时中运行此笔记本。在Colab中，选择* **运行时 > 更改运行时类型 > GPU**

###设置您的Google Cloud项目

**无论您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得$300的免费信用额，用于您的计算/存储成本。

2. [确保您的项目已启用计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下API：Vertex AI API、计算引擎API和云存储。](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您在本地运行此笔记本，您需要安装[Cloud SDK]((https://cloud.google.com/sdk))。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格，以确保Cloud SDK对本笔记本中的所有命令使用正确的项目。

**注意**：Jupyter运行以`!`为前缀的行作为shell命令，并插入以`$`为前缀的Python变量。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`来获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 地区

您还可以更改“REGION”变量，该变量用于整个笔记本的操作。以下是Vertex AI支持的地区。我们建议您选择距离您最近的地区。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能无法使用多区域存储桶来训练 Vertex AI。并非所有地区都支持所有Vertex AI服务。

了解更多有关[Vertex AI地区](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

时间戳

如果您在进行实时教程会话，可能会使用共享测试账户或项目。为了避免资源命名冲突，您可以为每个实例会话创建一个时间戳，然后将时间戳附加在您在本教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的谷歌云帐户

**如果您正在使用谷歌云笔记本**，您的环境已经通过验证。跳过这一步。

**如果您正在使用Colab**，运行下面的单元格，并按提示进行身份验证来通过oAuth验证您的帐户。

**否则**，请按照以下步骤操作：

在 Cloud Console 中，转到 [创建服务帐户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey) 页面。

**点击创建服务帐户**。

在**服务帐户名称**字段中输入名称，然后点击**创建**。

在**授予该服务帐户对项目的访问权限**部分，点击角色下拉列表。在过滤框中输入“Vertex”，然后选择**Vertex管理员**。在过滤框中输入“存储对象管理员”，然后选择**存储对象管理员**。

点击创建。包含您的密钥的 JSON 文件将下载到本地环境。

在下面的单元格中，输入您服务帐户密钥的路径作为 GOOGLE_APPLICATION_CREDENTIALS 变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = False
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        IS_COLAB = True
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用的是哪个笔记本环境，都需要执行以下步骤。**

当您为 Python 初始化 Vertex AI SDK 时，您需要指定一个云存储暂存桶。这个暂存桶是您数据集和模型资源所关联的数据在会话间保留的地方。

请在下方设置您的云存储桶的名称。存储桶的名称必须在所有 Google Cloud 项目中全局唯一，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_URI = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有当您的存储桶尚不存在时才运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查其内容来验证对云存储桶的访问。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。 
### 导入库和定义常量

In [ ]:
import google.cloud.aiplatform as aip
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text
from official.nlp import optimization  # to create AdamW optimizer

### 初始化用于Python的Vertex AI SDK

为您的项目和相应的存储桶初始化Python的Vertex AI SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

#### 设置硬件加速器

您可以为训练和预测设置硬件加速器。

将变量 `DEPLOY_GPU/DEPLOY_NGPU` 设置为使用支持 GPU 的容器映像以及分配给虚拟机实例的 GPU 数量。例如，要使用一个具有 4 个 Nvidia Telsa K80 GPU 的容器映像分配给每个虚拟机实例，您可以指定：

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)


否则，指定 `(None, None)` 以使用一个在 CPU 上运行的容器映像。

了解有关您地区的[硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

*注意*：TF 2.3 之前的 GPU 支持版本将无法在此教程中加载自定义模型。这是一个已知问题，在 TF 2.3 中已修复。这是由在服务函数中生成的静态图操作引起的。如果在您自己的自定义模型上遇到此问题，请使用在 TF 2.3 中带有 GPU 支持的容器映像。

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

设置预先构建的容器

为预测设置预先构建的Docker容器镜像。

获取最新列表，请查看[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2.5".replace(".", "-")

if TF[0] == "2":
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 设置机器类型

接下来，设置用于预测的机器类型。

- 将变量`DEPLOY_COMPUTE`设置为配置将用于预测的虚拟机的计算资源。
- `机器类型`
     - `n1-standard`：每个 vCPU 的内存为3.75GB。
     - `n1-highmem`：每个 vCPU 的内存为6.5GB。
     - `n1-highcpu`：每个 vCPU 的内存为0.9GB。
- `vCPUs`：数值范围为\[2, 4, 8, 16, 32, 64, 96\]

*注意：您也可以在训练和部署中使用 n2 和 e2 机器类型，但它们不支持GPU*。

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", DEPLOY_COMPUTE)

从TensorFlow Hub获取预训练的编码器嵌入模型

为了演示目的，本教程使用了从TensorFlow Hub（TFHub）获取的预训练的BERT编码器模型，对模型进行微调作为二元分类器，然后将其上传到`Vertex AI Model`资源。一旦您拥有了`Vertex AI Model`资源，该模型可以部署到`Vertex AI Endpoint`资源上。

### 下载预训练编码器模型

首先，您需要从TensorFlow Hub下载预训练的BERT编码器模型和对应的BERT文本预处理器。编码器和文本预处理器将会以TF.Keras层的形式下载。在本例中，为了完成模型，您需要创建一个使用下载的TFHub编码器和预处理器的`Functional`模型，并添加一个分类器将模型完成为一个二元分类器。

In [ ]:
inputs = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3", trainable=True
)
classifier = tf.keras.layers.Dense(
    1, activation="sigmoid", kernel_regularizer=tf.keras.regularizers.l2(0.001)
)

x = preprocess(inputs)
x = encoder(x)
outputs = classifier(x["pooled_output"])
tfhub_model = tf.keras.Model(inputs, outputs)

tfhub_model.summary()

### 下载数据集

从[互联网电影数据库](https://www.imdb.com/)下载[大型电影评论数据集](https://ai.stanford.edu/~amaas/data/sentiment/)。

In [ ]:
import shutil

url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file(
    "aclImdb_v1.tar.gz", url, untar=True, cache_dir=".", cache_subdir=""
)

dataset_dir = os.path.join(os.path.dirname(dataset), "aclImdb")

train_dir = os.path.join(dataset_dir, "train")

# remove unused folders to make it easier to load the data
remove_dir = os.path.join(train_dir, "unsup")
shutil.rmtree(remove_dir)

### 数据预处理

IMDB 数据集已经被划分为训练集和测试集，但缺乏验证集。为了创建一个验证集，您使用训练数据的80:20拆分，通过使用 validation_split 参数。

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32
seed = 42

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=seed,
)

class_names = raw_train_ds.class_names
train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)

val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=seed,
)

val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)

test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

### 编译模型

接下来，为后续的微调（迁移学习）进行编译。

In [ ]:
epochs = 3
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1 * num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(
    init_lr=init_lr,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    optimizer_type="adamw",
)

loss = tf.keras.losses.BinaryCrossentropy()
metrics = tf.metrics.BinaryAccuracy()

tfhub_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

训练模型

接下来，您训练（微调）模型。

In [ ]:
history = tfhub_model.fit(x=train_ds, validation_data=val_ds, epochs=epochs)

时代1/3
625/625 [==============================] - 5081秒 8秒/步 - 损失: 0.7344 - 二元准确率: 0.4785 - 验证损失: 0.7025 - 验证二元准确率: 0.4992
时代2/3
625/625 [==============================] - 5080秒 8秒/步 - 损失: 0.6966 - 二元准确率: 0.5167 - 验证损失: 0.6946 - 验证二元准确率: 0.5338
时代3/3
625/625 [==============================] - 预计时间: 0秒 - 损失: 0.6917 - 二元准确率: 0.5344

### 评估模型

接下来，评估模型。预计验证损失约为0.43。

In [ ]:
loss, accuracy = tfhub_model.evaluate(test_ds)

print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")

### 保存模型工件

此时，模型已存储在内存中。接下来，您可以将模型工件保存到云存储位置。

In [ ]:
MODEL_DIR = BUCKET_URI + "/model"
tfhub_model.save(MODEL_DIR)

### 将TensorFlow Hub模型上传至带有标准TensorFlow容器的“Vertex AI Model”资源

接下来，您将从TFHub模型中上传模型构件至带有标准TensorFlow容器的“Vertex AI Model”资源。

In [ ]:
DEPLOY_IMAGE = "us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-7:latest"

model_standard = aip.Model.upload(
    display_name="standard_" + TIMESTAMP,
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
)

print(model_standard)

### 将TensorFlow Hub模型上传到带有优化TensorFlow容器的“Vertex AI Model”资源

接下来，您将模型工件从TFHub模型上传到第二个带有TensorFlow Enterprise优化容器的“Vertex AI Model”资源：`us-docker.pkg.dev/vertex-ai-restricted/prediction/tf_opt-gpu.nightly:latest`。

模型应用了两种优化选项。

- *allow_precompilation* - 打开模型预编译以实现更好的性能。请注意，当具有新批量大小的第一个请求到达时，模型预编译会发生，并且在预编译完成后发送该请求的响应。为了减轻这种情况，请指定一个热身文件（请参阅此colab中较早的部分）。模型预编译适用于不同类型的模型，在大多数情况下对性能有积极影响。然而，我们建议您在生产环境之前尝试使用它来适应您的模型。

- *allow_precision_affecting_optimizations* - 启用影响精度的优化。在某些情况下，这会使模型运行速度显着提高，但会对模型预测能力造成非常小的损失。在使用此优化时，您应评估其对模型精度的影响。

In [ ]:
DEPLOY_IMAGE = "us-docker.pkg.dev/vertex-ai-restricted/prediction/tensorflow-enterprise-inference-engine-alpha:latest"
model_tfe_opt = aip.Model.upload(
    display_name="tfe_opt_" + TIMESTAMP,
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    serving_container_args=[
        "--allow_precompilation=true",
        "--allow_precision_affecting_optimizations=true",
    ],
)

print(model_tfe_opt)

创建两个公共 `Endpoint` 资源

您可以使用 `Endpoint.create()` 方法创建两个公共 `Endpoint` 资源。至少，您需要指定端点的显示名称。可选地，您可以指定项目和位置（地区）；否则，设置将继承在初始化 Vertex AI SDK 时使用 `init()` 方法设置的值。

在此示例中，指定了以下参数：

- `display_name`：`Endpoint` 资源的人类可读名称。
- `project`：您的项目 ID。
- `location`：您的地区。
- `labels`：（可选）`Endpoint` 的用户定义的元数据，以键/值对的形式。

此方法返回一个 `Endpoint` 对象。

了解有关 [Vertex AI Endpoints](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api) 的更多信息。

In [ ]:
endpoint_standard = aip.Endpoint.create(
    display_name="standard_" + TIMESTAMP,
    project=PROJECT_ID,
    location=REGION,
    labels={"your_key": "your_value"},
)

print(endpoint_standard)

endpoint_tfe_opt = aip.Endpoint.create(
    display_name="tfe_opt_" + TIMESTAMP,
    project=PROJECT_ID,
    location=REGION,
    labels={"your_key": "your_value"},
)

print(endpoint_tfe_opt)

部署`Model`资源到`Endpoint`资源。

接下来，您将标准和优化容器的两个实例部署到不同的公共`Vertex AI Endpoints`中。

*注意:* 在这个示例中，您在上一步将TFHub模型的部署容器指定为上传模型工件到`Vertex AI Model`资源。

在这个示例中，您将以最少数量的指定参数部署模型，具体如下：

- `model`：`Model`资源。
- `deployed_model_displayed_name`：部署模型实例的可读名称。
- `machine_type`：每个虚拟机实例的机器类型。

由于资源的需求，这可能需要几分钟的时间来完成。

In [ ]:
response = endpoint_standard.deploy(
    model=model_standard,
    deployed_model_display_name="standard_" + TIMESTAMP,
    machine_type=DEPLOY_COMPUTE,
)

print(endpoint_standard)

response = endpoint_tfe_opt.deploy(
    model=model_tfe_opt,
    deployed_model_display_name="tfe_opt_" + TIMESTAMP,
    machine_type=DEPLOY_COMPUTE,
)

print(endpoint_tfe_opt)

### 为预测准备测试数据

接下来，您要为预测准备测试数据。在这个例子中，您将使用合成数据。

In [ ]:
INSTANCES = ["This was the best movie ever", "Movie was boring"]

### 进行预测

现在您的`Model`资源已部署到`Endpoint`资源，您可以通过向Endpoint资源发送预测请求来进行在线预测。

#### 请求

预测请求中的每个实例都是以下形式的字典条目：

                        {input_name: content}

- `input_name`：底层模型的输入层的名称。
- `content`：作为1D Python列表的数据项。

由于`predict()`服务可以接受多个数据项（实例），您将将您的单个数据项发送为一个数据项的列表。最后一步是将实例列表打包成Google的protobuf格式 - 这是我们传递给`predict()`服务的内容。

#### 响应

从`predict()`调用的响应是一个Python字典，具有以下条目：

- `ids`：每个预测请求的内部分配的唯一标识符。
- `predictions`：每个类别标签的预测置信度，介于0和1之间。
- `deployed_model_id`：执行预测的已部署`Model`资源的Vertex AI标识符。

In [ ]:
serving_input = tfhub_model.input.name

# The format of each instance should conform to the deployed model's prediction input schema.
instances = [{serving_input: INSTANCES[0]}, {serving_input: INSTANCES[1]}]

prediction_standard = endpoint_standard.predict(instances=instances)

print(prediction_standard)

In [ ]:
# The format of each instance should conform to the deployed model's prediction input schema.
instances = [{serving_input: INSTANCES[0]}, {serving_input: INSTANCES[1]}]

prediction_tfe_opt = endpoint_tfe_opt.predict(instances=instances)

print(prediction_tfe_opt)

比较精度差异

最后，比较优化部署模型和未优化部署模型之间的精度差异。这个差异看起来非常微小。

In [ ]:
import numpy as np

abs(
    np.asarray(prediction_standard.predictions)
    - np.asarray(prediction_tfe_opt.predictions)
)

## 清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
delete_bucket = False
delete_model = True
delete_endpoint = True

if delete_endpoint:
    try:
        endpoint_standard.undeploy_all()
        endpoint_standard.delete()
        endpoint_tfe_opt.undeploy_all()
        endpoint_tfe_opt.delete()
    except Exception as e:
        print(e)

if delete_model:
    try:
        tfhub_model.delete()
    except Exception as e:
        print(e)

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}